# Supplemental information inline plots comparing models
## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, json, h5py
pj = os.path.join
# Less known modules, but very useful

from scripts.preprocess import write_conc_uM, read_conc_uM, string_to_tuple
from scripts.plotting import (
    change_log_ticks, 
    data_model_handles_legend, 
    standalone_legend, 
    corner_plot_mcmc, 
    plot_tcr_tcr_fits, 
    plot_autocorr, 
    standalone_parameter_values
)
from scripts.analysis import find_best_grid_point

In [ ]:
do_save_plots = False

root_dir = ".."
fig_dir = "panels_misc"
res_dir = pj(root_dir, "results", "for_plots")

In [ ]:
# Aesthetic parameters. Small scale figure.
# But scale everything up 2x times because otherwise too few axes tickes are put in by Matplotlib
# Will shrink to 50 % in InDesign to fit in the actual figure size. 
scaleup = 1.0

# Total of 4 panels + legend/cartoon + axes labels*4 + whitespace(0.5 in) = 7 inches, page width = 180 mm
# So panels = 5.5 / 5 inches seems reasonable
# Panel height: 2 rows for TCR/CAR, 2 rows for TCR/TCR, 1 for 6F, 2 for diagrams, total 170 mm = 6.7 inches
# Also 5 rows of labels = 1.25 in and say 0.5 in whitespace. So left with 5. / 7
panel_dimensions = {
    "panel_width": 5.5 / 5.0 * scaleup,     # inches
    "axes_label_width": 0.25 * scaleup,    # inches
    "panel_height": (5/7 + 0.05) * scaleup  # inches
}
panel_dimensions["legend_width"] = panel_dimensions["panel_width"]


# rcParams
plt.rcParams["font.size"] =  6. * scaleup  # Default, smallish
plt.rcParams["figure.dpi"] = 150.0
plt.rcParams["axes.labelpad"] = 0.5 * scaleup
plt.rcParams["axes.linewidth"] = 0.75 * scaleup     # edge line width
plt.rcParams["lines.linewidth"] = 1.5 * scaleup               # line width in points
plt.rcParams["lines.markersize"] = 2.5 * scaleup   # marker size, in points
for x in ["xtick.", "ytick."]:
    plt.rcParams[x + "major.size"] = 2.25 * scaleup
    plt.rcParams[x + "minor.size"] = 1.8 * scaleup
    plt.rcParams[x + "major.pad"] = 2.0 * scaleup
    plt.rcParams[x + "minor.pad"] = 1.9 * scaleup
    plt.rcParams[x + "minor.width"] = 0.5 * scaleup
    plt.rcParams[x + "major.width"] = 0.75 * scaleup
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

with open(pj(res_dir, "perturbations_palette.json"), "r") as h:
    perturbations_palette = json.load(h)
perturbations_palette["None"] = (0.0, 0.0, 0.0, 1.0)
sns.palplot(perturbations_palette.values())

# TCR/CAR model typical behaviours
Simple graphs. Two subplots, one legend. 

In [ ]:
def plot_model_behaviour(typical_curves_file, peak_info_file, model_name, pdims):
    # Read relevant data
    df_typical_curves = pd.read_hdf(typical_curves_file, key=model_name)
    df_peak_info_model = pd.read_hdf(peak_info_file, key="model_" + model_name)
    df_peak_info_data = pd.read_hdf(peak_info_file, key="data")
    
    # 2x2 plots, leave space for legend to the right of each row. 
    fig, axes = plt.subplots(2, 2)
    axes[0, 0].set_axis_off()  # Space for TCR/CAR diagram
    axes = [axes[0, 1], axes[1, 0], axes[1, 1]]
    # Keep the legend for someplace else ?
    fig.set_size_inches(pdims["panel_width"]*2 + pdims["axes_label_width"]*2,
                        pdims["panel_height"]*2 + pdims["axes_label_width"]*2)
    
    # Only plot 1 uM and 1 nM
    palet = {"1uM": perturbations_palette["None"], "1nM": perturbations_palette["AgDens"]}
    
    ## Part 1: typical model curves
    ax = axes[0]
    tau_range = df_typical_curves.columns.get_level_values("tau").values
    df_typical_curves = df_typical_curves.drop([0.1, 0.01], level="pulse_concentration", axis=0)
    pulse_range = df_typical_curves.index.get_level_values("pulse_concentration").values
    ax.axhline(1.0, ls="--", color="grey", lw=1.0*scaleup)
    for i in range(len(pulse_range)):
        conc_lbl = write_conc_uM(pulse_range[i])
        ax.plot(tau_range, df_typical_curves.iloc[i, :].values, 
                label=conc_lbl, color=palet[conc_lbl])
    ax.set(xlabel=r"TCR antigenicity, $\tau^T$ (s)", ylabel=r"FC$_{TCR \rightarrow CAR}$")
    ax.set_yscale("log", base=2)
    ax.set_xticks([0.0, 5.0, 10.0])
    ax.set_yticks([1/4, 1.0, 4.0])
    #ax.legend(title=r"TCR Ag Density", frameon=False, borderaxespad=0.1, 
    #          loc="upper left", bbox_to_anchor=(1.05, 1.0))
        
    ## Part 2: scaling of peak position with tau
    markers = {"1uM": "o", "1nM": "s"}
    model_color = (0.3,)*3 + (1.0,)
    ax = axes[1]
    peak_positions = df_peak_info_model["tau"].values
    pulse_conc_range = df_peak_info_model.index.get_level_values("pulse_concentration").values
    ax.plot(pulse_conc_range, peak_positions, label="Model", color=model_color)
    # Data
    for i, conc in enumerate(df_peak_info_data.index.get_level_values("TCR_Antigen_Density")):
        conc_um = read_conc_uM(conc)
        clr = palet.get(conc)
        lbl = "Data " + conc
        y = df_peak_info_data.loc[conc, "peptide"]
        ax.plot(conc_um, y, ls="none", marker=markers.get(conc), mfc=clr, mec=clr, 
               label=lbl, color=clr)
    # Labeling, etc.
    conc_lbl = r"TCR Ag density ($\mu$M)"
    ax.set(xlabel=conc_lbl, ylabel=r"Best antagonist $\tau^T$ (s)", xscale="log")
    ax.set_yticks([1.0, 2.0, 3.0, 4.0])
    ax.set_ylim(1.0, 4.0)
    ax.set_xticks([1e-3, 1e0])
    ax.set_xlim([5e-5, 2e0])
    
    ## Part 3: scaling of peak amplitude with tau
    # Plotting in log scale, changing y ticks labels later
    ax = axes[2]
    ax.axhline(0.0, ls="--", color="grey", lw=1.0)
    pulse_conc_range = df_peak_info_model.index.get_level_values("pulse_concentration").values
    min_outputs = df_peak_info_model["amplitude"].values
    
    ax.plot(pulse_conc_range, np.log2(min_outputs), label="Model", color=(0.3,)*3 + (1.0,))
    # Data
    for i, conc in enumerate(df_peak_info_data.index.get_level_values("TCR_Antigen_Density")):
        conc_um = read_conc_uM(conc)
        clr = palet.get(conc)
        lbl = "Data " + conc
        y = df_peak_info_data.loc[conc, "amplitude"]
        yerr = df_peak_info_data.loc[conc, "amplitude_CI"]
        ax.errorbar(conc_um, np.log2(y), yerr=yerr, 
                   ls="none", marker=markers.get(conc), mfc=clr, mec=clr, 
                   label=lbl, color=clr)

    ax.set(xlabel=conc_lbl, ylabel=r"FC$_{TCR \rightarrow CAR}$ at peak", xscale="log")
    ax.set_ylim([-3, 0.2])
    ax.set_yticks([-3, -2, -1, 0])
    ax.set_xticks([1e-3, 1e0])
    ax.set_xlim([5e-5, 2e0])
    change_log_ticks(ax, base=2, which="y")
    #ax.legend(frameon=False, borderaxespad=0.1, 
    #          loc="upper left", bbox_to_anchor=(1.05, 1.0))
    
    
    fig.tight_layout()
    return fig, axes

In [ ]:
def plot_model_behaviour_legends(typical_curves_file, peak_info_file, model_name, pdims):
    # Read relevant data
    df_typical_curves = pd.read_hdf(typical_curves_file, key=model_name)
    df_peak_info_model = pd.read_hdf(peak_info_file, key="model_" + model_name)
    df_peak_info_data = pd.read_hdf(peak_info_file, key="data")
    
    fig, axes = plt.subplots(2)
    axes = axes.flatten()
    fig.set_size_inches(pdims["legend_width"],  
                        pdims["panel_height"]*2 + pdims["axes_label_width"]*2)
    
    # First legend: model curves at different antigen densities
    # Only plot 1 uM and 1 nM as in previous function
    palet = {"1uM": perturbations_palette["None"], "1nM": perturbations_palette["AgDens"]}
    concentrations = ["1uM", "1nM"]
    labels1 = [r'1 $\mu$M', "1 nM"]
    handles1 = [mpl.lines.Line2D([0], [0], color=palet[c], lw=plt.rcParams["lines.linewidth"])
                for c in concentrations]
    axes[0].set_axis_off()
    axes[0].legend(handles1, labels1, title=r"TCR Ag Density", frameon=False, borderaxespad=0.1)
    
    # Second legend: model vs data. 
    markers = {"1uM": "o", "1nM": "s"}
    model_color = (0.3,)*3 + (1.0,)
    
    # Easier to dummy plot and use returned ErrorbarContainer
    axes[1].plot([1.0], [1.0], label="Model", color=model_color)
    for i, conc in enumerate(df_peak_info_data.index.get_level_values("TCR_Antigen_Density")):
        conc_um = read_conc_uM(conc)
        clr = palet.get(conc)
        lbl = "Data " + conc
        y = df_peak_info_data.loc[conc, "amplitude"]
        yerr = df_peak_info_data.loc[conc, "amplitude_CI"]
        axes[1].errorbar(conc_um, np.log2(y), yerr=yerr, 
                   ls="none", marker=markers.get(conc), mfc=clr, mec=clr, 
                   label=lbl, color=clr)
    handles2, labels2 = axes[1].get_legend_handles_labels()
    axes[1].clear()
    axes[1].set_axis_off()
    axes[1].legend(handles2, labels2, frameon=False, borderaxespad=0.1)
    
    fig.tight_layout()
    return fig, axes

In [ ]:
# Stand-alone legend
fig, axes = plot_model_behaviour_legends(pj(res_dir, "typical_tcr_car_model_curves.h5"), 
                     pj(res_dir, "peak_antagonism_tcr_car.h5"), "francois2013", panel_dimensions)
if do_save_plots:
    fig.savefig(pj(fig_dir, "legends_tcr_car_typical_antagonism.pdf"), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_model_behaviour(pj(res_dir, "typical_tcr_car_model_curves.h5"), 
                     pj(res_dir, "peak_antagonism_tcr_car.h5"), "francois2013", panel_dimensions)
if do_save_plots:
    fig.savefig(pj(fig_dir, "francois2013_tcr_car_typical_antagonism.pdf"), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_model_behaviour(pj(res_dir, "typical_tcr_car_model_curves.h5"), 
                     pj(res_dir, "peak_antagonism_tcr_car.h5"), "revised_akpr", panel_dimensions)
# We don't need the legends here, already plotted above.  
if do_save_plots:
    fig.savefig(pj(fig_dir, "revised_akpr_tcr_car_typical_antagonism.pdf"), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()